# Extracción del embedding

In [2]:
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1

mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device='cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(pretrained='vggface2').eval()

def get_embedding(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face = mtcnn(img)
    if face is None:
        raise ValueError("No se detectó rostro en la imagen.")
    face = face.unsqueeze(0)
    with torch.no_grad():
        embedding = model(face).detach().cpu().numpy()
    embedding = embedding / np.linalg.norm(embedding)
    return embedding

def main():
    embedding = get_embedding('rostro1.jpg') 
    np.save('embedding_registro.npy', embedding)
    print("Embedding guardado en embedding_registro.npy")

if __name__ == "__main__":
    main()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Comparación 

In [ ]:
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1

mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device='cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(pretrained='vggface2').eval()

def get_embedding(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face = mtcnn(img)
    if face is None:
        raise ValueError("No se detectó rostro en la imagen.")
    face = face.unsqueeze(0)
    with torch.no_grad():
        embedding = model(face).detach().cpu().numpy()
    embedding = embedding / np.linalg.norm(embedding)
    return embedding

def verify(embedding1, embedding2, threshold=0.6):
    cos_sim = np.dot(embedding1, embedding2.T)
    return cos_sim > (1 - threshold)

def main():
    emb_registro = np.load('embedding_registro.npy')
    emb_nuevo = get_embedding('rostro_0006.jpg')  

    if verify(emb_registro.flatten(), emb_nuevo.flatten()):
        print("¡Rostro verificado! Desbloqueo autorizado.")
    else:
        print("Rostro no coincide. Desbloqueo rechazado.")

if __name__ == "__main__":
    main()


¡Rostro verificado! Desbloqueo autorizado.


# Código alterno

In [29]:
import torch
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
import numpy as np

# Preprocesamiento que espera InceptionResnetV1
def preprocess_image_pytorch(image_path):
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),  # convierte a tensor y escala a [0,1]
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # normaliza a [-1,1]
    ])
    img = Image.open(image_path).convert('RGB')
    img_t = transform(img)
    img_t = img_t.unsqueeze(0)  # Añadir dimensión batch
    return img_t

# Extrae embedding facial usando el modelo
def get_face_embedding(model, image_path, device):
    img_tensor = preprocess_image_pytorch(image_path).to(device)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        embedding = model(img_tensor)
    embedding = embedding / embedding.norm()
    return embedding.cpu().numpy()

# Compara dos embeddings por distancia Euclidiana
def compare_faces(embedding1, embedding2, threshold=0.9):
    dist = np.linalg.norm(embedding1 - embedding2)
    if dist < threshold:
        print("Faces Matched")
    else:
        print("Faces are different")
    return dist

# --- Código principal ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = InceptionResnetV1(pretrained='vggface2').eval()

embedding1 = get_face_embedding(model, 'jimmy.jpg', device)
embedding2 = get_face_embedding(model, 'jimmy3.jpg', device)

distance = compare_faces(embedding1, embedding2)

print(f"Euclidean distance: {distance}")


Faces are different
Euclidean distance: 0.936867356300354
